In [1]:
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True))


BASE_URL = os.getenv("LITELLM_BASE_URL", "http://a6k2.dgx:34000/v1")
API_KEY = os.getenv("LITELLM_API_KEY", "")
MODEL_NAME = os.getenv("MODEL_NAME", "qwen3-32b")

In [2]:
from langchain_deepseek.chat_models import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal
import json

class ProblemSpec(BaseModel):
    """Problem specification for PINN"""
    problem_type: Literal["fluid_dynamics", "heat_transfer", "structural_mechanics", "epidemiology", "other"] = Field(description="Type of physical problem")
    confidence: float = Field(ge=0.0, le=1.0, description="Confidence in classification from 0 to 1")
    reasoning: str = Field(description="Explanation for the chosen problem type")

# Инициализация модели
llm = ChatDeepSeek(
    api_base=BASE_URL,
    base_url=BASE_URL,
    api_key=API_KEY,
    model=MODEL_NAME,
    streaming=False,
    timeout=120
)

# Промпт с экранированными фигурными скобками
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert in Physics-Informed Neural Networks (PINN). 
Analyze the problem description and determine its type.

result_output = {scheme}

Return ONLY valid JSON in this exact format."""),
    
    ("human", "PROBLEM DESCRIPTION: {query}")
])

def parse_response(response):
    """Pydantic автоматически парсит JSON"""
    content = response.content
    return ProblemSpec.model_validate_json(content)

chain = prompt | llm | parse_response

response = await chain.ainvoke({"scheme":ProblemSpec.model_json_schema(),
    "query": "We need to solve a problem of heat conduction in a rod. There are data points from sensors at several points, but they are noisy. The physics is described by the heat conduction equation. It is important to accurately observe the boundary conditions at the ends of the rod."
})

print(response)

APIConnectionError: Connection error.

In [13]:
response = await llm.ainvoke("Какая сегодня погода?")
print(response.content)

APIConnectionError: Connection error.

In [11]:
from langchain_deepseek.chat_models import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal

class ProblemSpec(BaseModel):
    """Problem specification for PINN"""
    problem_type: Literal["fluid_dynamics", "heat_transfer", "structural_mechanics", "epidemiology", "other"] = Field(description="Type of physical problem")
    confidence: float = Field(ge=0.0, le=1.0, description="Confidence in classification from 0 to 1")
    reasoning: str = Field(description="Explanation for the chosen problem type")

# Инициализация модели
llm = ChatDeepSeek(
    api_base=BASE_URL,
    base_url=BASE_URL,
    api_key=API_KEY,
    model=MODEL_NAME,
    streaming=False,
    timeout=120
)

# Промпт с экранированными фигурными скобками
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert in Physics-Informed Neural Networks (PINN). 
Analyze the problem description and determine its type.

result_output = {scheme}

Return ONLY valid JSON in this exact format."""),
    
    ("human", "PROBLEM DESCRIPTION: {query}")
])

def parse_response(response):
    """Pydantic автоматически парсит JSON"""
    content = response.content
    return ProblemSpec.model_validate_json(content)

chain = prompt | llm | parse_response

response = chain.invoke({"scheme":ProblemSpec.model_json_schema(),
    "query": "We need to solve a problem of heat conduction in a rod"
})

print(response)

APITimeoutError: Request timed out.

https://pydantic.dev/articles/llm-intro
https://dev.to/devasservice/a-practical-guide-on-structuring-llm-outputs-with-pydantic-50b4

In [ ]:
response

<coroutine object RunnableSequence.ainvoke at 0x000001AC92C852A0>